# Assignment 1 (60 points total)

You will train a convolutional neural network (aka ConvNet or CNN) to solve yet another image classification problem: the Tiny ImageNet dataset (200 classes, 100K training images, 10K validation images). Try to achieve as high accuracy as possible.

This exercise is close to what people do in real life. No toy architectures this time.

## Grading

* 11 points for the report.
* 5 points for using an **interactive** (don't reinvent the wheel with `plt.plot`) tool for viewing progress, for example TensorBoard.
* 9 points for a network that gets $\geq$25% accuracy on the private **test** set.
* Up to 35 points for accuracy up to 50%, issued linearly (i.e. 0 points for 25%, 7 points for 30%, 21 points for 40%, 35 points for $\geq$50%.

## Grading Explained

* *Private test set*: it's a part of the dataset like the validation set, but for which the ground truth labels are known only to us (you won't be able to evaluate your model on it). When grading, we will compute test accuracy by running your code that computes val accuracy, but having replaced the images in `'val/'` with the test set.
* *How to submit*:
  * **<font color="red">Read this in advance, don't leave until the last minute.</font> Wrong checkpoint submission = <font color="red">0 points for accuracy</font>. Be careful!**
  * After you've trained your network, [save weights](https://pytorch.org/tutorials/recipes/recipes/saving_and_loading_a_general_checkpoint.html) to "*checkpoint.pth*" with `model.state_dict()` and `torch.save()`.
  * Set `DO_TRAIN = False`, click "Restart and Run All" and make sure that your validation accuracy is computed correctly.
  * Compute the MD5 checksum for "*checkpoint.pth*" (e.g. run `!md5sum checkpoint.pth`) and paste it into "*solution.py*" (`get_checkpoint_metadata()`). You'll be penalized if this checksum doesn't match your submitted file.
  * Upload "*checkpoint.pth*" to Google Drive, copy the view-only link to it and paste it into "*solution.py*" as well.
  * Make sure "Restart and Run All" also works with `DO_TRAIN = True`, trains your model and saves `checkpoint.pth`.
  * <font color="red">Important</font>: At least several hours before the deadline, **upload "*solution.py*" [here](http://350e-83-69-192-100.ngrok.io/) and make sure you get a "👌"**.

* *Report*: PDF, free form; should mention:
  * Your history of tweaks and improvements. How you started, what you searched. (*I have analyzed these and those conference papers/sources/blog posts. I tried this and that to adapt them to my problem. ...*)
  * Which network architectures have you tried? Which of them didn't work, and can you guess why? What is the final one and why?
  * Same for the training method (batch size, optimization algorithm, number of iterations, ...): which and why?
  * Same for anti-overfitting (regularization) techniques. Which ones have you tried? What were their effects, and can you guess why?
  * **Most importantly**: deep learning insights you gained. Can you give several examples of how *exactly* experience from this exercise will affect you training your future neural nets? (tricks, heuristics, conclusions, observations)
  * **List all sources of code**.
* *Progress viewing tool*: support the report with screenshots of accuracy and loss plots (training and validation) over time.

## Restrictions

* No pretrained networks.
* Don't enlarge images (e.g. don't resize them to $224 \times 224$ or $256 \times 256$).

## Tips

* **One change at a time**: don't test several new things at once (unless you are super confident that they will work). Train a model, introduce one change, train again.
* Google a lot: try to reinvent as few wheels as possible. Harvest inspiration from PyTorch recipes, from GitHub, from blogs...
* Use GPU.
* Regularization is very important: L2, batch normalization, dropout, data augmentation...
* Pay much attention to accuracy and loss graphs (e.g. in TensorBoard). Track failures early, stop bad experiments early.
* 2-3 hours of training (in Colab) should be enough for most models, maybe 4-6 hours if you're experimenting.
* Save checkpoints every so often in case things go wrong (optimization diverges, Colab disconnects...).
* Don't use too large batches, they can be slow and memory-hungry. This is true for inference too.
* Also don't forget to use `torch.no_grad()` and `.eval()` during inference.

In [ ]:
!md5sum model.pth

md5sum: model.pth: No such file or directory


In [ ]:
# Determine the locations of auxiliary libraries and datasets.
# `AUX_DATA_ROOT` is where 'tiny-imagenet-2022.zip' is.

# Detect if we are in Google Colaboratory
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

from pathlib import Path
if IN_COLAB:
    google.colab.drive.mount("/content/drive")
    
    # Change this if you created the shortcut in a different location
    AUX_DATA_ROOT = Path("/content/drive/MyDrive/DL HW1")
    # /content/drive/MyDrive/DL HW1/tiny-imagenet-2020.zip
    assert AUX_DATA_ROOT.is_dir(), "Have you forgot to 'Add a shortcut to Drive'?"
    
    import sys
    sys.path.append(str(AUX_DATA_ROOT))
else:
    AUX_DATA_ROOT = Path(".")

Mounted at /content/drive


In [ ]:
# Imports
import torchvision
import torch
from torchvision import transforms,datasets
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
import os
from torchvision.utils import make_grid
from matplotlib.pyplot import imshow
from tqdm import tqdm
import numpy as np
from torch.optim import lr_scheduler
# Your solution
%load_ext autoreload
%autoreload 1


import solution

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [ ]:
# #!unzip -qq '/content/drive/MyDrive/DL HW1/tiny-imagenet-2020.zip'

# # Define main data directory
# DATA_DIR = '/content/drive/MyDrive/DL HW1/tiny-imagenet-200' # Original images come in shapes of [3,64,64]

# # Define training and validation data paths
# TRAIN_DIR = os.path.join(DATA_DIR, 'train') 
# VALID_DIR = os.path.join(DATA_DIR, 'val')

In [ ]:
# If `True`, will train the model from scratch and validate it.
# If `False`, instead of training will load weights from './checkpoint.pth'.
# When grading, we will test both cases.
DO_TRAIN = True

In [ ]:
# Put training and validation images in `./tiny-imagenet-200/train` and `./tiny-imagenet-200/val`:
if not Path("tiny-imagenet-200/train/class_000/00000.jpg").is_file():
    import zipfile
    with zipfile.ZipFile(AUX_DATA_ROOT / 'tiny-imagenet-2020.zip', 'r') as archive:
        archive.extractall()

In [ ]:
torch.cuda.empty_cache()

In [ ]:
# Initialize dataloaders
train_dataloader = solution.get_dataloader("./tiny-imagenet-200/", 'train',256)
val_dataloader   = solution.get_dataloader("./tiny-imagenet-200/", 'val',256)



# Initialize the raw model
model = solution.get_model().to(device)


writer = SummaryWriter()
images, labels = next(iter(train_dataloader))
images,labels = images.cuda(),labels.cuda()
grid = torchvision.utils.make_grid(images)
writer.add_image("images", grid)
writer.add_graph(model, images)
writer.close()


In [ ]:
PATH = "/content/drive/MyDrive/DL HW1/model.pth"

if DO_TRAIN:
    # Train from scratch
   # optimizer = solution.get_optimizer(model)
      optimizer = solution.get_optimizer(model)
      #solution.train_on_tinyimagenet(train_dataloader, val_dataloader, model, optimizer)
      solution.train_on_tinyimagenet(train_dataloader, val_dataloader, model,PATH, optimizer,num_epochs=40)
else:
    # Download the checkpoint and initialize model weights from it
    import urllib
    import subprocess

    penalize = False

    # Get your link and checksum
    claimed_md5_checksum, google_drive_link = solution.get_checkpoint_metadata()

    # Use your link to download "checkpoint.pth"
    !pip install -U gdown
    !gdown --id {urllib.parse.urlparse(google_drive_link).path.split('/')[-2]} -O model.pth

    try:
        # Compute the actual checksum
        real_md5_checksum = subprocess.check_output(
            ["md5sum", "model.pth"]).decode().split()[0]
    except subprocess.CalledProcessError as err:
        # Couldn't download or the filename isn't "checkpoint.pth"
        print(f"Wrong link or filename: {err}")
        penalize = True
    else:
        # The trained checkpoint is different from the one submitted
        if real_md5_checksum != claimed_md5_checksum:
            print("Checksums differ! Late submission?")
            penalize = True

    if penalize:
        print("🔫 Prepare the penalizer! 🔫")

    # Finally load weights
   # solution.load_weights(model, "./model_resnet50.pth")
    checkpoint = torch.load('./model.pth', map_location=device)
    model.load_state_dict(checkpoint["state_dict"])
  

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=runs

In [ ]:
# Classify some validation samples
import torch

example_batch, example_batch_labels = next(iter(val_dataloader))
model.eval()
with torch.no_grad():
  _, example_predicted_labels = solution.predict(model, example_batch).max(1)

print("Predicted class / Ground truth class")
for predicted, gt in list(zip(example_predicted_labels, example_batch_labels))[:15]:
    print("{:03d} / {:03d}".format(predicted, gt))

Predicted class / Ground truth class
157 / 157
024 / 024
141 / 141
020 / 020
011 / 011
145 / 024
007 / 007
171 / 171
092 / 152
090 / 090
138 / 175
192 / 192
137 / 137
073 / 073
021 / 126


In [ ]:
# Print validation accuracy
val_accuracy, _ = validate(val_dataloader, model)
val_accuracy *= 100
assert 1.5 <= val_accuracy <= 100.0
print("Validation accuracy: %.2f%%" % val_accuracy)

Validation accuracy: 49.03%
